In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import lightgbm as lgb

### read data

In [2]:
dataset = pd.read_csv(r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\dataset.csv")
additional_features = pd.read_pickle(r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\additional_features.pkl")
sentiment_lexicon_score = pd.read_pickle(r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\sentiment_lexicon_score.pkl")
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\summary_embeddings.npy"
with open(path, 'rb') as f:
    summary_embeddings = np.load(f)
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\review_embeddings.npy"
with open(path, 'rb') as f:
    review_embeddings = np.load(f)
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\keyword_embeddings.npy"
with open(path, 'rb') as f:
    keyword_embeddings = np.load(f)
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\tfidf.npy"
with open(path, 'rb') as f:
    tfidf = np.load(f)
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\tfidf_names.npy"
with open(path, 'rb') as f:
    tfidf_names = np.load(f, allow_pickle=True)
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\topic_words.npy"
with open(path, 'rb') as f:
    topic_words = np.load(f)
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\word_scores.npy"
with open(path, 'rb') as f:
    word_scores = np.load(f)
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\topic_similarity_scores.npy"
with open(path, 'rb') as f:
    topic_similarity_scores = np.load(f)

### prepare features

In [4]:
dataset['response'] = dataset.sentiment.map({"POS":1, "NEU":0,"NEG":-1})

In [5]:
summary_embeddings_df = pd.DataFrame(summary_embeddings, columns = list(map(lambda value: f"summary_embeddings_{value}", range(summary_embeddings.shape[1]))))
review_embeddings_df = pd.DataFrame(review_embeddings, columns = list(map(lambda value: f"review_embeddings_{value}", range(review_embeddings.shape[1]))))
keyword_embeddings_df = pd.DataFrame(keyword_embeddings, columns = list(map(lambda value: f"keyword_embeddings_{value}", range(keyword_embeddings.shape[1]))))
tfidf_df = pd.DataFrame(tfidf, columns = "tfidf_"+tfidf_names)
tfidf_cols = [col for col in tfidf_df.columns if col[6:] in topic_words.flatten()]
topic_similarity_scores_df = pd.DataFrame(topic_similarity_scores, columns = list(map(lambda value: f"topic_similarity_{value}", range(topic_similarity_scores.shape[1]))))

In [9]:
features = pd.concat([additional_features.merge(sentiment_lexicon_score, on = "reviewId"),summary_embeddings_df,review_embeddings_df,keyword_embeddings_df,tfidf_df[tfidf_cols],topic_similarity_scores_df], axis = 1)
features = features.drop("reviewId", axis = 1)

In [11]:
features

,num_words,num_uppercase_words,punc_!,"punc_""",punc_#,punc_$,punc_%,punc_&,punc_',punc_(,...,topic_similarity_26,topic_similarity_27,topic_similarity_28,topic_similarity_29,topic_similarity_30,topic_similarity_31,topic_similarity_32,topic_similarity_33,topic_similarity_34,topic_similarity_35
0,1439,231,0,17,0,1,0,1,39,26,...,0.076096,-0.016495,0.051254,0.181995,-0.002860,-0.002951,0.037847,-0.042878,0.012343,0.163936
1,746,113,0,6,0,0,0,0,9,13,...,-0.017875,0.049456,-0.028948,0.110548,0.030363,0.005043,0.063827,-0.019422,0.048254,0.136973
2,246,37,0,2,0,0,0,0,3,1,...,0.008039,-0.007581,0.026354,0.155227,0.059554,-0.006792,0.146159,0.088288,0.081141,0.240325
3,235,31,0,2,0,0,0,0,6,0,...,0.071355,0.120201,0.084645,0.166351,0.134689,-0.010713,0.155522,0.083599,0.104016,0.178283
4,211,41,0,0,0,0,0,0,7,2,...,0.069261,0.046579,0.070348,0.200227,0.095242,-0.011464,0.129631,0.016090,0.093500,0.198467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4435,2,0,0,0,0,0,0,0,0,0,...,0.128085,0.094547,0.154269,0.045453,-0.035820,0.043859,0.006198,0.112948,0.107429,0.029032
4436,61,14,0,0,0,0,0,0,0,0,...,0.187186,0.168873,0.128415,0.137136,0.106234,-0.116134,0.146911,0.232461,0.117452,0.118289
4437,135,12,2,0,0,0,0,0,2,0,...,0.079596,0.038898,0.148075,0.200458,0.158441,0.002689,0.167102,0.061767,0.103545,0.222701
4438,144,24,0,0,0,0,0,0,2,1,...,0.049273,-0.040922,0.088658,0.046652,0.033721,0.151388,0.299228,0.033641,0.111919,0.110183


### train test split

In [12]:
import lightgbm as lgb

In [32]:
# X_cols = [col for col in features if "tfidf" not in col]
# X_cols = ['sentiment_lexicon_score_summary',
#  'sentiment_lexicon_score_review']
# X_cols = [col for col in features if "summary_embeddings" in col]
# X_cols = ['num_words', 'num_uppercase_words', 'punc_!', 'punc_"', 'punc_#', 'punc_$', 'punc_%', 'punc_&', "punc_'", 'punc_(', 'punc_)', 'punc_*', 'punc_+', 'punc_,', 'punc_-', 'punc_.', 'punc_/', 'punc_:', 'punc_;', 'punc_<', 'punc_=', 'punc_>', 'punc_?', 'punc_@', 'punc_[', 'punc_\\', 'punc_]', 'punc_^', 'punc__', 'punc_`', 'punc_{', 'punc_|', 'punc_}', 'punc_~']
# X_cols = [col for col in features if "keyword_embeddings" in col]
# X_cols = [col for col in features if "review_embeddings" in col]
# X_cols = [col for col in features if "topic_similarity" in col]
# X_cols = tfidf_cols
X = features
y = dataset['response']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### model train

In [61]:
params = {
    "n_estimators":1000,
    'num_leaves': 163,
    'bagging_fraction': 0.80,
    'lambda_l1': 0.00019163693019842484,
    'lambda_l2': 0.0027009967266884796,
    'feature_fraction': 0.60,
    'boosting': 'gbdt',
    "learning_rate":0.01,
    "objective": "multiclass",
    "max_depth":  -1,
    "bagging_freq": 5 ,                  # resamples rows at every k-th iteration
    "force_col_wise":  True  ,                 # reduce memory cost
}

In [62]:
model = lgb.LGBMClassifier(**params)

In [24]:
punc_feature_names = {'punc_!': 'punc_0', 'punc_"': 'punc_1', 'punc_#': 'punc_2', 'punc_$': 'punc_3', 'punc_%': 'punc_4', 'punc_&': 'punc_5', "punc_'": 'punc_6', 'punc_(': 'punc_7', 'punc_)': 'punc_8', 'punc_*': 'punc_9', 'punc_+': 'punc_10', 'punc_,': 'punc_11', 'punc_-': 'punc_12', 'punc_.': 'punc_13', 'punc_/': 'punc_14', 'punc_:': 'punc_15', 'punc_;': 'punc_16', 'punc_<': 'punc_17', 'punc_=': 'punc_18', 'punc_>': 'punc_19', 'punc_?': 'punc_20', 'punc_@': 'punc_21', 'punc_[': 'punc_22', 'punc_\\': 'punc_23', 'punc_]': 'punc_24', 'punc_^': 'punc_25', 'punc__': 'punc_26', 'punc_`': 'punc_27', 'punc_{': 'punc_28', 'punc_|': 'punc_29', 'punc_}': 'punc_30', 'punc_~': 'punc_31'}
X.rename(columns = punc_feature_names, inplace = True)

In [63]:
model.fit(X_train,y_train, eval_set = (X_test, y_test), early_stopping_rounds = 10)

C:\Users\tanch\anaconda3\envs\ml\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] lambda_l1 is set=0.00019163693019842484, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00019163693019842484
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.0027009967266884796, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0027009967266884796
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[1]	valid_0's multi_logloss: 1.09454
[2]	valid_0's multi_logloss: 1.08901
[3]	valid_0's multi_logloss: 1.08361
[4]	valid_0's multi_logloss: 1.07874
[5]	valid_0's multi_logloss: 1.07319
[6]	valid_0's multi_logloss: 1.06832
[7]	valid_0's multi_logloss: 1.06338
[8]

[192]	valid_0's multi_logloss: 0.715742
[193]	valid_0's multi_logloss: 0.71525
[194]	valid_0's multi_logloss: 0.714494
[195]	valid_0's multi_logloss: 0.713862
[196]	valid_0's multi_logloss: 0.713321
[197]	valid_0's multi_logloss: 0.712668
[198]	valid_0's multi_logloss: 0.711975
[199]	valid_0's multi_logloss: 0.711252
[200]	valid_0's multi_logloss: 0.710428
[201]	valid_0's multi_logloss: 0.709877
[202]	valid_0's multi_logloss: 0.709137
[203]	valid_0's multi_logloss: 0.708508
[204]	valid_0's multi_logloss: 0.707817
[205]	valid_0's multi_logloss: 0.707176
[206]	valid_0's multi_logloss: 0.706535
[207]	valid_0's multi_logloss: 0.705741
[208]	valid_0's multi_logloss: 0.704987
[209]	valid_0's multi_logloss: 0.704524
[210]	valid_0's multi_logloss: 0.703722
[211]	valid_0's multi_logloss: 0.703078
[212]	valid_0's multi_logloss: 0.702435
[213]	valid_0's multi_logloss: 0.702089
[214]	valid_0's multi_logloss: 0.701392
[215]	valid_0's multi_logloss: 0.700649
[216]	valid_0's multi_logloss: 0.700152
[

[398]	valid_0's multi_logloss: 0.638556
[399]	valid_0's multi_logloss: 0.638396
[400]	valid_0's multi_logloss: 0.638202
[401]	valid_0's multi_logloss: 0.638043
[402]	valid_0's multi_logloss: 0.637941
[403]	valid_0's multi_logloss: 0.637783
[404]	valid_0's multi_logloss: 0.63761
[405]	valid_0's multi_logloss: 0.637405
[406]	valid_0's multi_logloss: 0.637165
[407]	valid_0's multi_logloss: 0.636957
[408]	valid_0's multi_logloss: 0.636653
[409]	valid_0's multi_logloss: 0.636349
[410]	valid_0's multi_logloss: 0.636205
[411]	valid_0's multi_logloss: 0.635868
[412]	valid_0's multi_logloss: 0.635599
[413]	valid_0's multi_logloss: 0.635292
[414]	valid_0's multi_logloss: 0.635336
[415]	valid_0's multi_logloss: 0.635115
[416]	valid_0's multi_logloss: 0.634952
[417]	valid_0's multi_logloss: 0.634881
[418]	valid_0's multi_logloss: 0.634753
[419]	valid_0's multi_logloss: 0.634527
[420]	valid_0's multi_logloss: 0.634331
[421]	valid_0's multi_logloss: 0.634256
[422]	valid_0's multi_logloss: 0.634094
[

LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting='gbdt',
               feature_fraction=0.6, force_col_wise=True,
               lambda_l1=0.00019163693019842484,
               lambda_l2=0.0027009967266884796, learning_rate=0.01,
               n_estimators=1000, num_leaves=163, objective='multiclass')

### feature importances

In [68]:
feature_importances = pd.Series(index = model.feature_name_, data = model.feature_importances_).sort_values(ascending = False)
feature_importances

sentiment_lexicon_score_summary    1711
sentiment_lexicon_score_review     1220
summary_embeddings_330              587
summary_embeddings_272              566
topic_similarity_35                 507
                                   ... 
tfidf_increase                        0
tfidf_inexpensive                     0
tfidf_infected                        0
tfidf_inferior                        0
punc_18                               0
Length: 2223, dtype: int32

In [69]:
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\feature_importances.pkl"
feature_importances.to_pickle(path)

### model training

In [64]:
y_pred = model.predict(X_test)

In [3]:
metrics.accuracy_score(y_test,y_pred)

NameError: name 'metrics' is not defined

In [73]:
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CS4022 - Social Media Mining\Assignments\Assignment 2 - Customer Review Mining Project\data\test_predictions.pkl"
pd.DataFrame({"y_pred":y_pred, "y_test":y_test}).to_pickle(path)